In [94]:
from amftrack.pipeline.development.high_mag_videos.kymo_class import *
from amftrack.pipeline.development.high_mag_videos.plot_data import (
    read_video_data,
    plot_summary
)
import os
import pandas as pd
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm import tqdm
%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
from pathlib import Path
from amftrack.pipeline.launching.run_super import run_parallel_flows
import re
from amftrack.util.dbx import upload_folders, upload, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, get_dropbox_folders, get_dropbox_video_folders
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/videos/MYRISTATE/DATA/2_weeks/"
upload_targ = "/DATA/MYRISTATE/DATA/2_weeks/"

In [96]:
test_name = glob(directory_targ + "*")[0]
imgs_address = test_name
print(test_name)

/gpfs/scratch1/shared/amftrackflow/videos/MYRISTATE/DATA/2_weeks/20230313_Plate031


In [105]:
plate_list = glob(directory_targ + "*_Plate*/*/")
datadict = {"tot_path_drop": plate_list}
dataframe = pd.DataFrame(data=datadict)

# print(dataframe['tot_path_drop'][0])

dataframe["video"] = [
    address[:-1].split(os.sep)[-1] for address in dataframe["tot_path_drop"]
]
dataframe["Plate number"] = [
    re.split("_|/", address.split("_")[-1])[0][5:]
    for address in dataframe["tot_path_drop"]
]
dataframe["video_int"] = [
    int(re.split("_|/", address[:-1])[-1]) for address in dataframe["tot_path_drop"]
]
dataframe["Date Imaged"] = [
    address.split("/")[-3][:8] for address in dataframe["tot_path_drop"]
]
dataframe["folder"] = [
    f"{row['Date Imaged']}_plate{row['Plate number']}_{row['video']}"
    for index, row in dataframe.iterrows()
]

print(dataframe["tot_path_drop"][0])
dataframe.sort_values(by="video")

/gpfs/scratch1/shared/amftrackflow/videos/MYRISTATE/DATA/2_weeks/20230313_Plate031/02/


,tot_path_drop,video,Plate number,video_int,Date Imaged,folder
2,/gpfs/scratch1/shared/amftrackflow/videos/MYRI...,01,031,1,20230313,20230313_plate031_01
0,/gpfs/scratch1/shared/amftrackflow/videos/MYRI...,02,031,2,20230313,20230313_plate031_02
4,/gpfs/scratch1/shared/amftrackflow/videos/MYRI...,03,031,3,20230313,20230313_plate031_03
3,/gpfs/scratch1/shared/amftrackflow/videos/MYRI...,04,031,4,20230313,20230313_plate031_04
1,/gpfs/scratch1/shared/amftrackflow/videos/MYRI...,05,031,5,20230313,20230313_plate031_05
5,/gpfs/scratch1/shared/amftrackflow/videos/MYRI...,06,031,6,20230313,20230313_plate031_06


In [106]:
data_table = []
for row in dataframe.iloc:
    excel_file = glob(
        str(Path(row["tot_path_drop"]).parent)
        + f'/*{row["Date Imaged"]}*{row["Plate number"]}.xl*'
    )
    if len(excel_file) > 0:
        data_table.append(excel_file[0])
    else:
        csv_file = glob(
            str(Path(row["tot_path_drop"]).parent)
            + f'/{row["Date Imaged"]}*{row["Plate number"]}.csv'
        )
        if len(csv_file) > 0:
            data_table.append(csv_file[0])
excel_addresses = pd.Series(data_table).unique()
excel_addresses

array(['/gpfs/scratch1/shared/amftrackflow/videos/MYRISTATE/DATA/2_weeks/20230313_Plate031/20230313_Plate031.csv'],
      dtype=object)

In [103]:
merge_frame = read_video_data(excel_addresses, dataframe)
merge_frame.tot_path[0]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 58.08it/s]


'/scratch1/shared/amftrackflow/videos/MYRISTATE/DATA/2_weeks/20230313_Plate031/02//'

In [104]:
download_frame = merge_frame.rename(columns={"tot_path": "folder"})
download_frame = download_frame.sort_values("unique_id")
download_frame = download_frame.reset_index(drop=True)
# dataframe_filtered = dataframe_filtered[dataframe_filtered['date_imaged'] == "20221026"]
# dataframe_filtered['plate_nr'].unique()
dataframe_filtered

,unique_id,video_int,treatment,strain,xpos,ypos,mode,binning,magnification,fps,time_(s),tot_path_drop,video_name,imaging_day,plate_id,tot_path,video_id,plate_nr
0,20230313_Plate031_2,1,0Myr,C2,28520,80140,BF,2,50.0,20.0,30.0,/gpfs/scratch1/shared/amftrackflow/videos/MYRI...,02,20230313,20230313_Plate031,/scratch1/shared/amftrackflow/videos/MYRISTATE...,2,31
1,20230313_Plate031_5,2,0Myr,C2,27140,80180,BF,2,50.0,20.0,10.0,/gpfs/scratch1/shared/amftrackflow/videos/MYRI...,05,20230313,20230313_Plate031,/scratch1/shared/amftrackflow/videos/MYRISTATE...,5,31
2,20230313_Plate031_1,3,0Myr,C2,32600,86020,BF,2,50.0,20.0,10.0,/gpfs/scratch1/shared/amftrackflow/videos/MYRI...,01,20230313,20230313_Plate031,/scratch1/shared/amftrackflow/videos/MYRISTATE...,1,31
3,20230313_Plate031_4,4,0Myr,C2,35220,78840,BF,2,50.0,20.0,10.0,/gpfs/scratch1/shared/amftrackflow/videos/MYRI...,04,20230313,20230313_Plate031,/scratch1/shared/amftrackflow/videos/MYRISTATE...,4,31
4,20230313_Plate031_3,5,0Myr,C2,38860,93780,BF,2,50.0,20.0,30.0,/gpfs/scratch1/shared/amftrackflow/videos/MYRI...,03,20230313,20230313_Plate031,/scratch1/shared/amftrackflow/videos/MYRISTATE...,3,31
5,20230313_Plate031_6,6,0Myr,C2,39560,93820,BF,2,50.0,20.0,10.0,/gpfs/scratch1/shared/amftrackflow/videos/MYRI...,06,20230313,20230313_Plate031,/scratch1/shared/amftrackflow/videos/MYRISTATE...,6,31


In [107]:
# FINAL CHECK FOR SOURCE FOLDER AND UPLOAD FOLDER

# Please make sure that the upload folder is correct,
# as the program WILL overwrite that which is already there.
print(directory_targ)
print(upload_targ)

/gpfs/scratch1/shared/amftrackflow/videos/MYRISTATE/DATA/2_weeks/
/DATA/MYRISTATE/DATA/2_weeks/


In [108]:
nr_parallel = np.min([len(dataframe.index), 16])

run_parallel_flows(
    "flux_extract.py",
    [directory_targ, 15, 0.95, 0.001, 60, upload_targ],
    dataframe_filtered,
    nr_parallel,
    "1:00:00",
    "flux_extract",
)

/home/svstaalduine/bash/job.sh
Sending jobs with id 1688478924896177678
6


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3015215


Upload the proper excel files to the analysis folder.

In [56]:
parent_folder = []
data_table = []
magnification = []
fps = []
mode = []
binning = []


for row in dataframe.iloc:
    parent_folder.append(str(Path(row["tot_path_drop"]).parent))
    excel_file = glob(
        str(Path(row["tot_path_drop"]).parent)
        + f'/*{row["Date Imaged"]}*{row["Plate number"]}.xl*'
    )
    if len(excel_file) > 0:
        data_table.append(excel_file[0])
        excel_table = pd.read_excel(excel_file[0])
        magnification.append(
            excel_table[
                excel_table["Unnamed: 0"].str.contains(
                    row["video_name"], case=False, na=False
                )
            ]["Magnification"].iloc[0]
        )
        fps.append(
            excel_table.loc[
                excel_table["Unnamed: 0"].str.contains(
                    row["video_name"], case=False, na=False
                )
            ]["FPS"].iloc[0]
        )
        mode.append(
            excel_table.loc[
                excel_table["Unnamed: 0"].str.contains(
                    row["video_name"], case=False, na=False
                )
            ]["Bright-field (BF)\nor\nFluorescence (F)"].iloc[0]
        )
        if "Binned (Y/N)" in excel_table:
            binning.append(
                excel_table.loc[
                    excel_table["Unnamed: 0"].str.contains(
                        row["video_name"], case=False, na=False
                    )
                ]["Binned (Y/N)"].iloc[0]
            )
        else:
            binning.append("N")
    else:
        csv_file = glob(
            str(Path(row["tot_path_drop"]).parent)
            + f'/{row["Date Imaged"]}*{row["Plate number"]}.csv'
        )
        if len(csv_file) > 0:
            data_table.append(csv_file[0])
            df_comma = pd.read_csv(csv_file[0], nrows=1, sep=",")
            df_semi = pd.read_csv(csv_file[0], nrows=1, sep=";")
            if df_comma.shape[1] > df_semi.shape[1]:
                csv_table = pd.read_csv(csv_file[0], sep=",")
            else:
                csv_table = pd.read_csv(csv_file[0], sep=";")

            magnification.append(
                csv_table[csv_table["video"] == int(row["video_name"])]["Lens"].iloc[0]
            )
            fps.append(
                csv_table.loc[
                    csv_table["video"] == int(row["video_name"].split("_")[-1])
                ]["fps"].iloc[0]
            )
            mode.append(
                csv_table.loc[
                    csv_table["video"] == int(row["video_name"].split("_")[-1])
                ]["Illumination"].iloc[0]
            )
            binning.append(
                csv_table.loc[
                    csv_table["video"] == int(row["video_name"].split("_")[-1])
                ]["Binned"].iloc[0]
            )
        else:
            print("Halp! No datatable found! Pls add the excel file to the folders.")

dataframe["fps"] = fps
dataframe["magnification"] = magnification
dataframe["mode"] = mode
dataframe["parent_folder"] = [
    os.path.relpath(address, directory_targ) for address in dataframe["tot_path_drop"]
]
dataframe["data_table"] = data_table
dataframe["binned"] = binning

dataframe = dataframe.sort_values(by="video_name", ignore_index=True)
# dataframe = dataframe[dataframe['video_nr']== 9]
# dataframe = dataframe[dataframe['plate_nr'] == '046']
# print(len(dataframe))
# print(data_table)


if dataframe.isnull().values.any():
    print("Found NaNs in the excel files! Blame the experimentalists.")
    dataframe = dataframe.interpolate(method="pad", limit_direction="forward")
if dataframe.isnull().values.any():
    raise ("This excel sheet is unworkable, please ask the responsible person")

In [28]:
excel_series = pd.Series(data_table).unique()
excel_series

array(['/gpfs/scratch1/shared/amftrackflow/videos/MYRISTATE/DATA/2_weeks/20230313_Plate031/20230313_Plate031.csv'],
      dtype=object)

In [19]:
for xl_address in excel_series:
    file_name = "Analysis/" + os.path.relpath(xl_address, directory_targ)
    print(upload_targ + file_name)
    upload(xl_adress, upload_targ + file_name)

/DATA/FLUORESCENCE/DATA_NileRed/Analysis/20230123_Plate545/Fluorescence_ex_20230123_Plate545.xlsx
2023-06-20 11:12:22,949-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-20 11:12:23,475-[INFO]- dropbox:474 -> Request to files/upload
/DATA/FLUORESCENCE/DATA_NileRed/Analysis/20221108_Plate462/Fluorescence_ex_20221108_plate462.xlsx
2023-06-20 11:12:24,281-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-20 11:12:24,452-[INFO]- dropbox:474 -> Request to files/upload
/DATA/FLUORESCENCE/DATA_NileRed/Analysis/20230125_Plate532/Fluorescence_ex_20230125_Plate532.xlsx
2023-06-20 11:12:24,970-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-20 11:12:25,158-[INFO]- dropbox:474 -> Request to files/upload
/DATA/FLUORESCENCE/DATA_NileRed/Analysis/20221109_Plate462/Fluorescence_ex_20221109_plate462.xlsx
2023-06-20 11:12:25,825-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-20 11:12:26,029-[INFO]- dropbox:474 -> Request to files/upload
/DATA/FLUORESCENCE/DATA_NileRed/Anal